In [1]:
import astropy.units as u
from tdsat_telescope import load_telescope_parameters
from tdsat_sensitivity import src_rate, bgd_sky_qe_rate, bgd_electronics, calc_exposure
from tdsat_neff import get_neff
from bbmag import bb_abmag_fluence
import numpy as np
from matplotlib import pyplot as plt
from bbmag import bb_abmag_fluence, bb_abmag

from astropy.visualization import quantity_support
import matplotlib
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)

In [2]:
siglimit = 10. # sigma
read_noise = 3. # Read noise for two frames

point_drift = 1*u.arcsec # To be added to the PSF in quadrature
point_jitter = 5*u.arcsec


# Transmission efficiency
trans_eff = (0.975)**8 # from Jim. 

print('Transmission efficiency: {}'.format(trans_eff))

# Load telescope info
config=11
diameter, qe, psf_size, pixel_size, efficiency = load_telescope_parameters(config)

area = np.pi * (diameter*0.5)**2
area_onaxis = np.pi * (0.5*24.2*u.cm)**2
efficiency = area_onaxis / area

print('Effective area (used) {}'.format(area*efficiency))

# Add in fuzz...
psf_size = (psf_size**2 + point_drift**2 + point_jitter**2)**0.5


print('Effective PSF size {}'.format(psf_size))
# Get the number of effective background pixels
neff = get_neff(psf_size, pixel_size)
print('Number of effective bgd pixels: {}'.format(neff))
print()


Transmission efficiency: 0.8166518036622619
Telescope Configuration 11
Name: Small Focal Plane CMOS
Entrance Pupil diameter 26.0 cm
Optical Efficiency 0.7893639053254439
PSF FWHM 4.288 arcsec
Pixel size 6.4 arcsec
Effective Aperture 23.1 cm
Effective Area 419.09631397051237 cm2
Effective area (used) 459.9605804120816 cm2
Effective PSF size 6.6623527375845235 arcsec
Number of effective bgd pixels: 2.990374488245114



In [3]:
# Band one
light=True

center = 208
width = 53

bandone=[center - 0.5*width, center+0.5*width]*u.nm


for swiftmag in np.arange(18, 25, 0.05):
    band1_fluence, band2_fluence = bb_abmag_fluence(swiftmag=swiftmag*u.ABmag, bbtemp=10e3*u.K, light=light)
    band1_rate = trans_eff  * efficiency * area * band1_fluence

    if (band1_rate.value < 1.0):
        bbmag1, foo = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=10e3*u.K, bandone = bandone)
        print('Band 1 Photometric Zero Point (Swift UVW2): {}'.format(swiftmag))
        print('Band 1 Photometric Zero Point: {}'.format(bbmag1))


        break
print()       
for swiftmag in np.arange(18, 25, 0.05):
    band1_fluence, band2_fluence = bb_abmag_fluence(swiftmag=swiftmag*u.ABmag, bbtemp=20e3*u.K, light=light)
    band1_rate = trans_eff  * efficiency * area * band1_fluence

    if (band1_rate.value < 1.0):
        bbmag1, foo = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=20e3*u.K, bandone = bandone)
        print('Band 1 Photometric Zero Point (Swift UVW2): {}'.format(swiftmag))
        print('Band 1 Photometric Zero Point: {}'.format(bbmag1))
        break


Band 1 Photometric Zero Point (Swift UVW2): 20.950000000000042
Band 1 Photometric Zero Point: 20.841517476112692 mag(AB)

Band 1 Photometric Zero Point (Swift UVW2): 20.75000000000004
Band 1 Photometric Zero Point: 20.73219194328544 mag(AB)


In [4]:
light=True

center = 284
width = 68
bandtwo=[center - 0.5*width, center+0.5*width]*u.nm


for swiftmag in np.arange(18, 25, 0.05):
    band1_fluence, band2_fluence = bb_abmag_fluence(swiftmag=swiftmag*u.ABmag, bbtemp=10e3*u.K, light=light)
    band2_rate = trans_eff * efficiency * area * band2_fluence

    if (band2_rate.value < 1.0):
        foo, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=10e3*u.K, bandtwo = bandtwo)
        print('Band 2 Photometric Zero Point: {}'.format(swiftmag))
        print('Band 2 Photometric Zero Point: {}'.format(bbmag2))
        break
        
print()       
for swiftmag in np.arange(18, 25, 0.05):
    band1_fluence, band2_fluence = bb_abmag_fluence(swiftmag=swiftmag*u.ABmag, bbtemp=20e3*u.K, light=light)
    band2_rate = trans_eff * efficiency * area * band2_fluence

    if (band2_rate.value < 1.0):
        foo, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=20e3*u.K, bandtwo = bandtwo)
        print('Band 2 Photometric Zero Point: {}'.format(swiftmag))
        print('Band 2 Photometric Zero Point: {}'.format(bbmag2))
        break

Band 2 Photometric Zero Point: 21.750000000000053
Band 2 Photometric Zero Point: 20.638294603048834 mag(AB)

Band 2 Photometric Zero Point: 20.650000000000038
Band 2 Photometric Zero Point: 20.573919385664702 mag(AB)


In [7]:
light=True
siglimit=10


exp1 = []
sm = []

# Compute sky background
bgd_ph_rate = bgd_sky_qe_rate(low_zodi=True,
                              diameter=diameter,
                              pixel_size=pixel_size, real_red=True, light=light)
bgd_band1 = trans_eff * efficiency * bgd_ph_rate
print('Sky background rate per pixel band1: {}'.format(bgd_band1))

bgd_ph_rate = bgd_sky_qe_rate(low_zodi=True,
                              diameter=diameter,
                              qe_band=2,
                              pixel_size=pixel_size, real_red=True, light=light)
bgd_band2 = trans_eff  * efficiency * bgd_ph_rate
print('Sky background rate per pixel band2: {}'.format(bgd_band2))


center = 208
width = 53
bandone=[center - 0.5*width, center+0.5*width]*u.nm

center = 284
width = 68
bandtwo=[center - 0.5*width, center+0.5*width]*u.nm



print()
for siglimit in [5, 10]:
    print()
    for nframes, explimit in enumerate([300, 600, 900]):
        for swiftmag in np.arange(20, 24, 0.05):
            band1_fluence, foo = bb_abmag_fluence(swiftmag=swiftmag*u.ABmag, bbtemp=20e3*u.K, light=light)
            band1_rate = trans_eff  * efficiency * area * band1_fluence
            exposure1 = calc_exposure(siglimit, band1_rate, bgd_band1,read_noise*(nframes+1)**0.5, neff)
            if (exposure1.value > explimit):
                bbmag1, foo = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=20e3*u.K, bandone = bandone)
                print('Band 1 {}-s {}-σ magnitude limit: {}'.format(explimit, siglimit, bbmag1))
                break

print()

for siglimit in [5, 10]:
    print()
    for nframes, explimit in enumerate([300, 600, 900, 1200]):
        for swiftmag in np.arange(20, 24, 0.05):
            foo, band2_fluence = bb_abmag_fluence(swiftmag=swiftmag*u.ABmag, bbtemp=20e3*u.K, light=light)
            band2_rate = trans_eff  * efficiency * area * band2_fluence
            exposure2 = calc_exposure(siglimit, band2_rate, bgd_band2,read_noise*(nframes+1)**0.5, neff)
            if (exposure2.value > explimit):
                foo, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=20e3*u.K, bandtwo = bandtwo)
                print('Band 2 {}-s {}-σ magnitude limit: {}'.format(explimit, siglimit, bbmag2))
                break

Sky background rate per pixel band1: 0.030369732491096913 1 / s
Sky background rate per pixel band2: 0.2478588509265617 1 / s


Band 1 300-s 5-σ magnitude limit: 22.632191943285438 mag(AB)
Band 1 600-s 5-σ magnitude limit: 23.08219194328544 mag(AB)
Band 1 900-s 5-σ magnitude limit: 23.382191943285452 mag(AB)

Band 1 300-s 10-σ magnitude limit: 21.532191943285422 mag(AB)
Band 1 600-s 10-σ magnitude limit: 22.13219194328543 mag(AB)
Band 1 900-s 10-σ magnitude limit: 22.432191943285435 mag(AB)


Band 2 300-s 5-σ magnitude limit: 21.873919385664692 mag(AB)
Band 2 600-s 5-σ magnitude limit: 22.273919385664698 mag(AB)
Band 2 900-s 5-σ magnitude limit: 22.523919385664705 mag(AB)
Band 2 1200-s 5-σ magnitude limit: 22.723919385664708 mag(AB)

Band 2 300-s 10-σ magnitude limit: 20.92391938566468 mag(AB)
Band 2 600-s 10-σ magnitude limit: 21.423919385664686 mag(AB)
Band 2 900-s 10-σ magnitude limit: 21.67391938566469 mag(AB)
Band 2 1200-s 10-σ magnitude limit: 21.873919385664692 mag(AB)
